In [7]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [20]:
import duckdb
import pandas as pd

import cmf
from cmf.helpers import selector

In [14]:
ch_selector = selector(
    table="companieshouse.companies",
    fields=["company_name", "company_number", "postcode"],
)

ch_raw = cmf.query(selector=ch_selector, return_type="pandas")

In [16]:
ch_raw.head(3)
type(ch_raw.data_sha1[0])
ch_raw.info(memory_usage='deep')

,data_sha1,companieshouse_companies_company_name,companieshouse_companies_company_number,companieshouse_companies_postcode
0,b'<\xee\x08\x93\x07\x10M\xdd\x1a\xbc\xc3*T\xa9...,00498943 LIMITED,00498943,WA14 5ER
1,"b""\xdb\xe0\x9b\xbe\x06V\x8c\r'\x17;E\x0e\xe3\x...",029-001 LTD,14045431,EC2M 2RH
2,b'\xb2\xde\xed\x87*\xa2\xbbo\xe1)\x05jZ\xc0J\x...,0800GASMAN LIMITED,11332800,IP14 1NW


bytes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5499110 entries, 0 to 5499109
Data columns (total 4 columns):
 #   Column                                   Dtype 
---  ------                                   ----- 
 0   data_sha1                                object
 1   companieshouse_companies_company_name    object
 2   companieshouse_companies_company_number  object
 3   companieshouse_companies_postcode        object
dtypes: object(4)
memory usage: 1.4 GB


In [17]:
ch_pyarrow = ch_raw.convert_dtypes(dtype_backend="pyarrow")
ch_pyarrow.head(3)
type(ch_pyarrow.data_sha1[0])
ch_pyarrow.info(memory_usage='deep')

,data_sha1,companieshouse_companies_company_name,companieshouse_companies_company_number,companieshouse_companies_postcode
0,b'<\xee\x08\x93\x07\x10M\xdd\x1a\xbc\xc3*T\xa9...,00498943 LIMITED,00498943,WA14 5ER
1,"b""\xdb\xe0\x9b\xbe\x06V\x8c\r'\x17;E\x0e\xe3\x...",029-001 LTD,14045431,EC2M 2RH
2,b'\xb2\xde\xed\x87*\xa2\xbbo\xe1)\x05jZ\xc0J\x...,0800GASMAN LIMITED,11332800,IP14 1NW


bytes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5499110 entries, 0 to 5499109
Data columns (total 4 columns):
 #   Column                                   Dtype          
---  ------                                   -----          
 0   data_sha1                                object         
 1   companieshouse_companies_company_name    string[pyarrow]
 2   companieshouse_companies_company_number  string[pyarrow]
 3   companieshouse_companies_postcode        string[pyarrow]
dtypes: object(1), string[pyarrow](3)
memory usage: 585.6 MB


In [18]:
df = duckdb.sql("""
    select * from ch_pyarrow;
""").df()

df.head(3)
type(df.data_sha1[0])

,data_sha1,companieshouse_companies_company_name,companieshouse_companies_company_number,companieshouse_companies_postcode
0,"[60, 238, 8, 147, 7, 16, 77, 221, 26, 188, 195...",00498943 LIMITED,00498943,WA14 5ER
1,"[219, 224, 155, 190, 6, 86, 140, 13, 39, 23, 5...",029-001 LTD,14045431,EC2M 2RH
2,"[178, 222, 237, 135, 42, 162, 187, 111, 225, 4...",0800GASMAN LIMITED,11332800,IP14 1NW


bytearray

In [21]:
df2 = duckdb.sql("""
    select * from ch_raw;
""").arrow().to_pandas(types_mapper=pd.ArrowDtype)



df2.head(3)
type(df2.data_sha1[0])

,data_sha1,companieshouse_companies_company_name,companieshouse_companies_company_number,companieshouse_companies_postcode
0,b'<\xee\x08\x93\x07\x10M\xdd\x1a\xbc\xc3*T\xa9...,00498943 LIMITED,00498943,WA14 5ER
1,"b""\xdb\xe0\x9b\xbe\x06V\x8c\r'\x17;E\x0e\xe3\x...",029-001 LTD,14045431,EC2M 2RH
2,b'\xb2\xde\xed\x87*\xa2\xbbo\xe1)\x05jZ\xc0J\x...,0800GASMAN LIMITED,11332800,IP14 1NW


bytes